In [ ]:
import keras
from keras.layers import Dense, Conv2D, BatchNormalization, Activation, MaxPooling2D, Add, Input, ZeroPadding2D, AveragePooling2D, Flatten
from keras.models import Model
import os
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import Adam

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
def identity_block(X, f, filters):
    F1, F2, F3 = filters

    X_shortcut = X

    # First component of main path
    X = Conv2D(F1, (1, 1))(X)
    X = BatchNormalization(axis=3)(X)
    X = Activation('relu')(X)

    # Second component of main path
    X = Conv2D(F2, (f, f), padding='same')(X)
    X = BatchNormalization(axis=3)(X)
    X = Activation('relu')(X)

    # Third component of main path
    X = Conv2D(F3, (1, 1))(X)
    X = BatchNormalization(axis=3)(X)

    # Add shortcut to the main path
    X = Add()([X, X_shortcut])
    X = Activation('relu')(X)

    return X

In [ ]:
def convolutional_block(X, f, filters, s=2):
    F1, F2, F3 = filters

    X_shortcut = X

    # First component of main path
    X = Conv2D(F1, (1, 1), strides=(s, s))(X)
    X = BatchNormalization(axis=3)(X)
    X = Activation('relu')(X)

    # Second component of main path
    X = Conv2D(F2, (f, f), padding='same')(X)
    X = BatchNormalization(axis=3)(X)
    X = Activation('relu')(X)

    # Third component of main path
    X = Conv2D(F3, (1, 1))(X)
    X = BatchNormalization(axis=3)(X)

    # Shortcut path
    X_shortcut = Conv2D(F3, (1, 1), strides=(s, s))(X_shortcut)
    X_shortcut = BatchNormalization(axis=3)(X_shortcut)

    # Add shortcut to the main path
    X = Add()([X, X_shortcut])
    X = Activation('relu')(X)

    return X

In [ ]:
def ResNet50(input_shape=(224, 224, 3), classes=1000):
    # Define the input as a tensor with shape input_shape
    X_input = Input(input_shape)

    # Zero-padding
    X = ZeroPadding2D((3, 3))(X_input)

    # Stage 1
    X = Conv2D(64, (7, 7), strides=(2, 2))(X)
    X = BatchNormalization(axis=3)(X)
    X = Activation('relu')(X)
    X = MaxPooling2D((3, 3), strides=(2, 2))(X)

    # Stage 2
    X = convolutional_block(X, f=3, filters=[64, 64, 256], s=1)
    X = identity_block(X, 3, [64, 64, 256])
    X = identity_block(X, 3, [64, 64, 256])

    # Stage 3
    X = convolutional_block(X, f=3, filters=[128, 128, 512], s=2)
    X = identity_block(X, 3, [128, 128, 512])
    X = identity_block(X, 3, [128, 128, 512])
    X = identity_block(X, 3, [128, 128, 512])

    # Stage 4
    X = convolutional_block(X, f=3, filters=[256, 256, 1024], s=2)
    X = identity_block(X, 3, [256, 256, 1024])
    X = identity_block(X, 3, [256, 256, 1024])
    X = identity_block(X, 3, [256, 256, 1024])
    X = identity_block(X, 3, [256, 256, 1024])
    X = identity_block(X, 3, [256, 256, 1024])

    # Stage 5
    X = convolutional_block(X, f=3, filters=[512, 512, 2048], s=2)
    X = identity_block(X, 3, [512, 512, 2048])
    X = identity_block(X, 3, [512, 512, 2048])

    # AVGPOOL
    X = AveragePooling2D(pool_size=(7, 7))(X)

    # Output layer
    X = Flatten()(X)
    X = Dense(classes, activation='softmax')(X)

    # Create model
    model = Model(inputs=X_input, outputs=X)

    return model

model = ResNet50()

model.summary()


Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 224, 224, 3)]        0         []                            
                                                                                                  
 zero_padding2d (ZeroPaddin  (None, 230, 230, 3)          0         ['input_1[0][0]']             
 g2D)                                                                                             
                                                                                                  
 conv2d (Conv2D)             (None, 112, 112, 64)         9472      ['zero_padding2d[0][0]']      
                                                                                                  
 batch_normalization (Batch  (None, 112, 112, 64)         256       ['conv2d[0][0]']          

In [ ]:
train_dir = '/content/drive/MyDrive/Data_Int/BreakHis_v1_100/output/train'
val_dir = '/content/drive/MyDrive/Data_Int/BreakHis_v1_100/output/val'
test_dir = '/content/drive/MyDrive/Data_Int/BreakHis_v1_100/output/test'

In [ ]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

val_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

In [ ]:
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical')

validation_generator = val_datagen.flow_from_directory(
    val_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical')

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical')

Found 1455 images belonging to 2 classes.
Found 415 images belonging to 2 classes.
Found 211 images belonging to 2 classes.


In [ ]:
model = ResNet50(input_shape=(224, 224, 3), classes=2)
model.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // validation_generator.batch_size,
    epochs=100)

Epoch 1/10
45/45 [==============================] - 1068s 23s/step - loss: 0.5403 - accuracy: 0.7702 - val_loss: 0.6207 - val_accuracy: 0.6927
Epoch 2/10
45/45 [==============================] - 54s 1s/step - loss: 0.4199 - accuracy: 0.8039 - val_loss: 0.8617 - val_accuracy: 0.6927
Epoch 3/10
45/45 [==============================] - 53s 1s/step - loss: 0.4200 - accuracy: 0.8208 - val_loss: 1.1299 - val_accuracy: 0.6927
Epoch 4/10
45/45 [==============================] - 51s 1s/step - loss: 0.3853 - accuracy: 0.8377 - val_loss: 1.7110 - val_accuracy: 0.6901
Epoch 5/10
45/45 [==============================] - 53s 1s/step - loss: 0.3731 - accuracy: 0.8306 - val_loss: 1.8679 - val_accuracy: 0.6875
Epoch 6/10
45/45 [==============================] - 50s 1s/step - loss: 0.3711 - accuracy: 0.8271 - val_loss: 2.3964 - val_accuracy: 0.6875
Epoch 7/10
45/45 [==============================] - 52s 1s/step - loss: 0.4076 - accuracy: 0.8180 - val_loss: 2.0716 - val_accuracy: 0.6875
Epoch 8/10
45/45 

In [ ]:
test_loss, test_acc = model.evaluate(test_generator, steps=test_generator.samples // test_generator.batch_size)
print('Test accuracy:', test_acc)
print('Test loss:', test_loss)

6/6 [==============================] - 120s 24s/step - loss: 1.2446 - accuracy: 0.6667
Test accuracy: 0.6666666865348816
Test loss: 1.244571328163147


In [ ]:
model.save('/content/drive/My Drive/path_to_save_model/resnet50_model.h5')

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
